<a href="https://colab.research.google.com/github/richie0115/PD/blob/main/%E7%B4%AF%E4%BA%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
!pip install arch
!pip install pmdarima
!pip install tensorflow
!pip install scikit-learn
!pip install statsmodels

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model

from transformers import pipeline

In [ ]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional, Conv1D, MaxPooling1D, Flatten
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model


In [ ]:

# LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, :-1])
        y.append(data[i, -1])
    return np.array(X), np.array(y)

def train_lstm(X, y, sequence_length=60):
    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, pd.Series):
        y = y.values

    data = np.concatenate((X, y.reshape(-1, 1)), axis=1)

    # 創建序列數據
    X_seq, y_seq = create_sequences(data, sequence_length)
    # 分割訓練集和測試集
    X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

    # 建立LSTM模型
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # 定義早停和學習率衰減回調
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    # 訓練模型
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping, reduce_lr])

    return model, sequence_length, X_test, y_test

# CNN+LSTM
def train_cnn_lstm(X, y):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(np.concatenate((X, y.values.reshape(-1, 1)), axis=1))

    def create_sequences(data, seq_length):
        X, y = [], []
        for i in range(seq_length, len(data)):
            X.append(data[i-seq_length:i, :-1])
            y.append(data[i, -1])
        return np.array(X), np.array(y)

    sequence_length = 60
    X_seq, y_seq = create_sequences(scaled_data, sequence_length)
    X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping, reduce_lr])
    return model, scaler, sequence_length, X_test, y_test

# 隨機森林
def train_random_forest(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestRegressor(n_estimators=300, max_depth=10, min_samples_split=5, min_samples_leaf=4, bootstrap=True))
    ])
    pipeline.fit(X_train, y_train)
    return pipeline, X_test, y_test

# SVR
def train_svr(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    svr = SVR(kernel='linear', C=0.1, gamma='scale', epsilon=0.01)
    svr.fit(X_train_scaled, y_train)
    return svr, scaler, X_test_scaled, y_test

# XGBoost
def train_xgboost(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    xgb_model = xgb.XGBRegressor(
        objective='reg:squarederror', booster='gbtree', eval_metric='rmse',
        n_estimators=500, learning_rate=0.1, max_depth=5, min_child_weight=10,
        subsample=0.9, colsample_bytree=0.8, gamma=0, alpha=0.01, reg_lambda=1, random_state=42
    )
    xgb_model.fit(X_train, y_train)
    return xgb_model, X_test, y_test

# Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim):
        super(TransformerModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

def train_transformer(X, y):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    input_dim = X_train.shape[1]
    model = TransformerModel(input_dim=input_dim).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 50
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
    return model, scaler, X_test_tensor, y_test_tensor

# ARIMA+GARCH
def train_arima_garch(df, target):
    arima_order = (5, 1, 0)
    model_arima = ARIMA(df[target], order=arima_order)
    model_fit_arima = model_arima.fit()

    resid = model_fit_arima.resid
    model_garch = arch_model(resid, vol='Garch', p=1, q=1)
    model_fit_garch = model_garch.fit(disp='off')

    forecast_arima = model_fit_arima.forecast(steps=1)
    forecast_garch = model_fit_garch.forecast(horizon=1)
    next_return = forecast_arima.values[-1] + forecast_garch.variance.values[-1]

    return next_return

# 每個模型的呼叫
lstm_model, lstm_seq_len, X_test_lstm, y_test_lstm = train_lstm(df[features], df[target])
cnn_lstm_model, cnn_lstm_scaler, cnn_lstm_seq_len, X_test_cnn_lstm, y_test_cnn_lstm = train_cnn_lstm(df[features], df[target])
rf_model, X_test_rf, y_test_rf = train_random_forest(df[features], df[target])
svr_model, svr_scaler, X_test_svr, y_test_svr = train_svr(df[features], df[target])
xgb_model, X_test_xgb, y_test_xgb = train_xgboost(df[features], df[target])
transformer_model, transformer_scaler, X_test_transformer, y_test_transformer = train_transformer(df[features], df[target])
next_return_arima_garch = train_arima_garch(df, target)

# 預測
# LSTM
lstm_pred = lstm_model.predict(X_test_lstm[-1].reshape(1, lstm_seq_len, X_test_lstm.shape[2]))[0][0]

# CNN+LSTM
cnn_lstm_pred = cnn_lstm_model.predict(X_test_cnn_lstm[-1].reshape(1, cnn_lstm_seq_len, X_test_cnn_lstm.shape[2]))[0][0]

# RF
rf_pred = rf_model.predict(X_test_rf.iloc[-1].values.reshape(1, -1))[0]

# SVR
svr_pred = svr_model.predict(X_test_svr[-1].reshape(1, -1))[0]
# svr_pred = svr_model.predict(X_test_svr.iloc[-1].reshape(1, -1))[0]

# XGBoost
xgb_pred = xgb_model.predict(X_test_xgb[-1].reshape(1, -1))[0]
# xgb_pred = xgb_model.predict(X_test_xgb.iloc[-1].reshape(1, -1))[0]

# Transformer
transformer_pred = transformer_model(X_test_transformer[-1].view(1, -1)).cpu().detach().numpy()[0][0]


# 主迴圈
iterations = 100  # 定義迴圈次數
results = []

for i in range(iterations):
    print(f'第 {i+1} 次實驗：')

    # 讀取原始Excel文件並複製
    df_original = pd.read_excel('AMD.xlsx')
    df = df_original.copy()

    # 隨機刪除數據
    if len(df) > 2500:
        delete_point = random.randint(2501, len(df))
        df = df.iloc[:delete_point]

    df.ffill(inplace=True)
    df.bfill(inplace=True)

    # 重新計算日期轉換和 last_date_numeric
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date_numeric'] = df['Date'].dt.strftime('%Y%m%d').astype(int)
    last_date_numeric = df['Date_numeric'].iloc[-1]

    # 定義特徵和目標變數
    features = ["Date_numeric", "Open", "High", "Low", "Close", "Adj Close", "Volume", "MA14",
                "RSI14", "RSI28", "RSI50", "EMA30", "EMA60", "EMA90",
                "AROON_UP25", "AROON_DOWN25", "AROON_UP50", "AROON_DOWN50",
                "current_ratio", "quick_ratio", "debt_ratio", "gross_margin",
                "operating_margin", "roe", "net_profit_margin",
                "diluted_earing_per_share", "operating_cash_flow_margin",
                "cash_flow_coverage_ratio", "DAILYRETURN", "WMA14", "WMA20", "WMA200", "std20", "UPPER_BAND", "LOWER_BAND", "MACD", "RVI"]

    target = 'DAILYRETURN'
    X = df[features]
    y = df[target]

    # 訓練和預測
    lstm_model, lstm_seq_len, X_test_lstm, y_test_lstm = train_lstm(X, y)
    cnn_lstm_model, cnn_lstm_scaler, cnn_lstm_seq_len, X_test_cnn_lstm, y_test_cnn_lstm = train_cnn_lstm(X, y)
    rf_model, X_test_rf, y_test_rf = train_random_forest(X, y)
    svr_model, svr_scaler, X_test_svr, y_test_svr = train_svr(X, y)
    xgb_model, X_test_xgb, y_test_xgb = train_xgboost(X, y)
    transformer_model, transformer_scaler, X_test_transformer, y_test_transformer = train_transformer(X, y)
    next_return_arima_garch = train_arima_garch(df, target)

    # 預測
    lstm_pred = lstm_model.predict(X_test_lstm[-1].reshape(1, lstm_seq_len, X_test_lstm.shape[2]))[0][0]
    cnn_lstm_pred = cnn_lstm_model.predict(X_test_cnn_lstm[-1].reshape(1, cnn_lstm_seq_len, X_test_cnn_lstm.shape[2]))[0][0]
    rf_pred = rf_model.predict(X_test_rf.iloc[-1].values.reshape(1, -1))[0]
    svr_pred = svr_model.predict(X_test_svr[-1].reshape(1, -1))[0]
    xgb_pred = xgb_model.predict(X_test_xgb[-1].reshape(1, -1))[0]
    transformer_pred = transformer_model(X_test_transformer[-1].view(1, -1)).cpu().detach().numpy()[0][0]

    # 紀錄結果，包括最後一項的 Date_numeric
    result = f'第 {i+1} 次實驗結果 (最後一項 Date_numeric: {last_date_numeric}):\n'
    result += f'LSTM模型預測結果: {lstm_pred*100}%\n'
    result += f'CNN+LSTM模型預測結果: {cnn_lstm_pred*10}%\n'
    result += f'隨機森林模型預測結果: {rf_pred*100}%\n'
    result += f'SVR模型預測結果: {svr_pred*100}%\n'
    result += f'XGBoost模型預測結果: {xgb_pred*100}%\n'
    result += f'Transformer模型預測結果: {transformer_pred*100}%\n'
    result += f'ARIMA+GARCH模型預測結果: {next_return_arima_garch*100}%\n'

    results.append(result)
    print(result)

# 將所有結果寫入txt檔案
with open('experiment_results.txt', 'w') as f:
    for result in results:
        f.write(result + '\n')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 38s 437ms/step - loss: 0.0864 - mae: 0.2327 - val_loss: 0.0112 - val_mae: 0.0928 - learning_rate: 0.0010
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 19s 333ms/step - loss: 0.0287 - mae: 0.1325 - val_loss: 0.0046 - val_mae: 0.0567 - learning_rate: 0.0010
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 212ms/step - loss: 0.0121 - mae: 0.0838 - val_loss: 0.0014 - val_mae: 0.0304 - learning_rate: 0.0010
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 21s 219ms/step - loss: 0.0057 - mae: 0.0561 - val_loss: 0.0014 - val_mae: 0.0305 - learning_rate: 0.0010
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 220ms/step - loss: 0.0033 - mae: 0.0445 - val_loss: 9.4558e-04 - val_mae: 0.0232 - learning_rate: 0.0010
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 21s 233ms/step - loss: 0.0024 - mae: 0.0364 - val_loss: 8.9371e-04 - val_mae: 0.0225 - learning_rate: 0.0010
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 222ms/step - loss: 0.0021 - mae: 0.0331 - val_loss: 8.9420e-04 - val_mae: 0.0227 - le

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 15s 158ms/step - loss: 0.1659 - mae: 0.3156 - val_loss: 0.0430 - val_mae: 0.2026 - learning_rate: 0.0010
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 132ms/step - loss: 0.0230 - mae: 0.1190 - val_loss: 0.0018 - val_mae: 0.0318 - learning_rate: 0.0010
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 107ms/step - loss: 0.0066 - mae: 0.0641 - val_loss: 0.0094 - val_mae: 0.0896 - learning_rate: 0.0010
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 133ms/step - loss: 0.0049 - mae: 0.0545 - val_loss: 0.0089 - val_mae: 0.0871 - learning_rate: 0.0010
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 140ms/step - loss: 0.0051 - mae: 0.0543 - val_loss: 0.0027 - val_mae: 0.0441 - learning_rate: 0.0010
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 119ms/step - loss: 0.0039 - mae: 0.0470 - val_loss: 0.0016 - val_mae: 0.0314 - learning_rate: 0.0010
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 7s 131ms/step - loss: 0.0045 - mae: 0.0465 - val_loss: 0.0016 - val_mae: 0.0296 - learning_rate: 0.00

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001574. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 19s 169ms/step - loss: 0.1816 - mae: 0.3343 - val_loss: 0.0510 - val_mae: 0.2220 - learning_rate: 0.0010
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 158ms/step - loss: 0.0357 - mae: 0.1487 - val_loss: 0.0022 - val_mae: 0.0348 - learning_rate: 0.0010
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - loss: 0.0078 - mae: 0.0685 - val_loss: 0.0249 - val_mae: 0.1521 - learning_rate: 0.0010
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 124ms/step - loss: 0.0058 - mae: 0.0595 - val_loss: 0.0144 - val_mae: 0.1136 - learning_rate: 0.0010
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 12s 146ms/step - loss: 0.0049 - mae: 0.0532 - val_loss: 0.0068 - val_mae: 0.0752 - learning_rate: 0.0010
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 161ms/step - loss: 0.0044 - mae: 0.0490 - val_loss: 0.0028 - val_mae: 0.0459 - learning_rate: 0.0010
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 7s 124ms/step - loss: 0.0041 - mae: 0.0463 - val_loss: 0.0014 - val_mae: 0.0295 - learning_rate: 0

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001576. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
第 21 次實驗結果 (最後一項 Date_numeric: 20240131):
LSTM模型預測結果: 0.21569801028817892%
CNN+LSTM模型預測結果: 3.051261007785797%
隨機森林模型預測結果: -13.00408428441333%
SVR模型預測結果: -3.4170393113631596%
XGBoost模型預測結果: -3.20315845310688%
Transformer模型預測結果: -0.3451840952038765%
ARIMA+GARCH模型預測結果: [-0.11025385]%

第 22 次實驗：


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
